In [4]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda")

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

question = "how many humans are in the picture?"
inputs = processor(raw_image, question, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


1


In [5]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
!pip install bert-score rouge-score nltk sacrebleu --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# !pip install git+https://github.com/neulab/BARTScore.git --quiet

In [7]:
!git clone https://github.com/neulab/BARTScore.git



fatal: destination path 'BARTScore' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
setup_code = """
from setuptools import setup, find_packages

setup(
    name='bart_score',
    version='0.1.0',
    description='BARTScore: Evaluating Text Generation with BART',
    author='Neulab',
    url='https://github.com/neulab/BARTScore',
    packages=find_packages(),
    install_requires=[
        'torch>=1.7.0',
        'transformers>=4.0.0',
        'scipy',
        'numpy',
        'tqdm',
    ],
    classifiers=[
        'Programming Language :: Python :: 3',
        'License :: OSI Approved :: MIT License',
    ],
    python_requires='>=3.6',
)
"""
with open("BARTScore/setup.py", "w") as f:
    f.write(setup_code)


In [9]:
!pip install ./BARTScore

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing ./BARTScore
  Preparing metadata (setup.py) ... done
  Created wheel for bart_score: filename=bart_score-0.1.0-py3-none-any.whl size=5317 sha256=17596198578b8cd43ef40e8e83f697fa9ec149746abf7010e91c9b9f07e39b9f
  Stored in directory: /tmp/pip-ephem-wheel-cache-0wosw4j5/wheels/98/01/d3/10d9ca6b2865ade955e5f3c526e4a9b480ab24cea364919527
Successfully built bart_score
  Attempting uninstall: bart_score
    Found existing installation: bart_score 0.1.0
    Uninstalling bart_score-0.1.0:
      Successfully uninstalled bart_score-0.1.0


In [ ]:
# # import os
# import json
# import csv
# import pickle
# from PIL import Image
# from tqdm import tqdm
# from sklearn.metrics import accuracy_score, f1_score
# from nltk.translate.bleu_score import sentence_bleu
# from rouge_score import rouge_scorer
# from BARTScore.bart_score import BARTScorer
# from bert_score import score as bert_score
# import numpy as np
# from concurrent.futures import ThreadPoolExecutor, as_completed

# # --- Define Paths ---
# json_folder_path = "/kaggle/input/test-dataset/test_dataset"
# image_base_path = "/kaggle/input/abo-small/small"
# all_qa_data_path = "/kaggle/working/all_qa_data1.pkl"
# progress_file = "/kaggle/working/progress1.json"

# # from your_model_library import model, processor

# # ---- LOAD CACHED DATA OR BUILD ----
# if os.path.exists(all_qa_data_path):
#     with open(all_qa_data_path, "rb") as f:
#         all_qa_data = pickle.load(f)
# else:
#     print("🔄 Processing Q&A data from JSON files...")
#     all_qa_data = []

#     def process_json(json_file):
#         results = []
#         if not json_file.endswith(".json"):
#             return results
#         json_path = os.path.join(json_folder_path, json_file)
#         try:
#             with open(json_path, "r") as f:
#                 data = json.load(f)
#         except Exception:
#             return results

#         raw_path = data["image_path"].replace("\\", "/")
#         path_parts = raw_path.split("/")[-2:]
#         image_rel_path = "/".join(path_parts)
#         full_img_path = os.path.join(image_base_path, image_rel_path)

#         if not os.path.exists(full_img_path):
#             return results

#         try:
#             _ = Image.open(full_img_path).convert("RGB")  # preload test
#         except Exception:
#             return results

#         for idx, qa_pair in enumerate(data.get("qa_pairs", [])):
#             question = qa_pair.get("question", "").strip()
#             answer = qa_pair.get("answer", "").strip()
#             if question and answer:
#                 results.append((json_file, idx, full_img_path, question, answer))
#         return results

#     with ThreadPoolExecutor() as executor:
#         futures = [executor.submit(process_json, jf) for jf in os.listdir(json_folder_path)]
#         for future in tqdm(as_completed(futures), total=len(futures), desc="Processing JSON files"):
#             all_qa_data.extend(future.result())

#     with open(all_qa_data_path, "wb") as f:
#         pickle.dump(all_qa_data, f)
#     print("✅ Q&A data saved to disk.")

# # ---- LOAD PROGRESS ----
# if os.path.exists(progress_file):
#     try:
#         with open(progress_file, "r") as f:
#             completed_set = set(tuple(x) for x in json.load(f))
#         print(f"📘 Resuming from saved progress ({len(completed_set)} Q&A pairs done).")
#     except json.JSONDecodeError as e:
#         print(f"Error reading progress: {e}")
#         completed_set = set()
# else:
#     completed_set = set()
#     print("Progress file does not exist. Starting fresh.")

# # ---- EVALUATION FUNCTION ----
# def evaluate_qa_pairs(pairs, description, csv_filename):
#     y_true, y_pred = [], []
#     all_preds, all_gts = [], []
#     bleu_scores, rougeL_scores, bart_scores = [], []

#     rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
#     bart_scorer = BARTScorer(device="cuda", checkpoint="facebook/bart-large-cnn")

#     with open(f"/kaggle/working/{csv_filename}", 'w', newline='') as csvfile:
#         fieldnames = ['json_file', 'question_id', 'question', 'ground_truth', 'predicted_answer']
#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#         writer.writeheader()

#         for json_file, idx, img_path, question, ground_truth in tqdm(pairs, desc="🔍 Evaluating"):
#             if (json_file, idx) in completed_set:
#                 continue

#             try:
#                 raw_image = Image.open(img_path).convert("RGB")
#                 inputs = processor(raw_image, question, return_tensors="pt").to("cuda")
#                 out = model.generate(**inputs)
#                 answer = processor.decode(out[0], skip_special_tokens=True)
#             except Exception as e:
#                 print(f"[ERROR] {json_file} Q{idx+1} → {e}")
#                 answer = "ERROR"

#             gt = ground_truth.strip().lower()
#             pred = answer.strip().lower()
#             y_true.append(gt)
#             y_pred.append(pred)
#             all_preds.append(pred)
#             all_gts.append(gt)

#             bleu_scores.append(sentence_bleu([gt.split()], pred.split()))
#             rougeL_scores.append(rouge.score(gt, pred)['rougeL'].fmeasure)
#             bart_scores.append(bart_scorer.score([pred], [gt])[0])

#             writer.writerow({
#                 'json_file': json_file,
#                 'question_id': idx,
#                 'question': question,
#                 'ground_truth': ground_truth,
#                 'predicted_answer': answer
#             })

#             completed_set.add((json_file, idx))
#             with open(progress_file, "w") as f:
#                 json.dump(list(completed_set), f)

#     # BERTScore (batched)
#     print("📏 Calculating BERTScore...")
#     P, R, F1 = bert_score(all_preds, all_gts, lang='en', verbose=True)
#     bert_P = P.numpy()
#     bert_R = R.numpy()
#     bert_F1 = F1.numpy()

#     acc = accuracy_score(y_true, y_pred)
#     f1 = f1_score(y_true, y_pred, average="macro")

#     print(f"\n✅ Evaluation complete: {description}")
#     print(f"Accuracy:       {acc:.4f}")
#     print(f"F1 Score:       {f1:.4f}")
#     print(f"BLEU:           {np.mean(bleu_scores):.4f}")
#     print(f"ROUGE-L:        {np.mean(rougeL_scores):.4f}")
#     print(f"BARTScore:      {np.mean(bart_scores):.4f}")
#     print(f"BERTScore (P/R/F1): {np.mean(bert_P):.4f} / {np.mean(bert_R):.4f} / {np.mean(bert_F1):.4f}")

# # ---- MAIN EXECUTION ----
# filtered_qa_data = [item for item in all_qa_data if (item[0], item[1]) not in completed_set]
# evaluate_qa_pairs(filtered_qa_data, "Full Dataset", "full_dataset_results1.csv")


In [ ]:
# import os
# import json
# import csv
# import pickle
# from PIL import Image
# from tqdm import tqdm
# from sklearn.metrics import accuracy_score, f1_score
# from nltk.translate.bleu_score import sentence_bleu
# from rouge_score import rouge_scorer
# from BARTScore.bart_score import BARTScorer
# from bert_score import score as bert_score
# import numpy as np

In [ ]:
# from concurrent.futures import ThreadPoolExecutor, as_completed

# # --- Define Paths ---
# json_folder_path = "/kaggle/input/test-dataset/test_dataset"
# image_base_path = "/kaggle/input/abo-small/small"
# all_qa_data_path = "/kaggle/working/all_qa_data1.pkl"
# progress_file = "/kaggle/working/progress1.json"

# # from your_model_library import model, processor

# # ---- LOAD CACHED DATA OR BUILD ----
# if os.path.exists(all_qa_data_path):
#     with open(all_qa_data_path, "rb") as f:
#         all_qa_data = pickle.load(f)
# else:
#     print("🔄 Processing Q&A data from JSON files...")
#     all_qa_data = []

#     def process_json(json_file):
#         results = []
#         if not json_file.endswith(".json"):
#             return results
#         json_path = os.path.join(json_folder_path, json_file)
#         try:
#             with open(json_path, "r") as f:
#                 data = json.load(f)
#         except Exception:
#             return results

#         raw_path = data["image_path"].replace("\\", "/")
#         path_parts = raw_path.split("/")[-2:]
#         image_rel_path = "/".join(path_parts)
#         full_img_path = os.path.join(image_base_path, image_rel_path)

#         if not os.path.exists(full_img_path):
#             return results

#         try:
#             _ = Image.open(full_img_path).convert("RGB")  # preload test
#         except Exception:
#             return results

#         for idx, qa_pair in enumerate(data.get("qa_pairs", [])):
#             question = qa_pair.get("question", "").strip()
#             answer = qa_pair.get("answer", "").strip()
#             if question and answer:
#                 results.append((json_file, idx, full_img_path, question, answer))
#         return results

#     with ThreadPoolExecutor() as executor:
#         futures = [executor.submit(process_json, jf) for jf in os.listdir(json_folder_path)]
#         for future in tqdm(as_completed(futures), total=len(futures), desc="Processing JSON files"):
#             all_qa_data.extend(future.result())

#     with open(all_qa_data_path, "wb") as f:
#         pickle.dump(all_qa_data, f)
#     print("✅ Q&A data saved to disk.")

# # ---- LOAD PROGRESS ----
# if os.path.exists(progress_file):
#     try:
#         with open(progress_file, "r") as f:
#             completed_set = set(tuple(x) for x in json.load(f))
#         print(f"📘 Resuming from saved progress ({len(completed_set)} Q&A pairs done).")
#     except json.JSONDecodeError as e:
#         print(f"Error reading progress: {e}")
#         completed_set = set()
# else:
#     completed_set = set()
#     print("Progress file does not exist. Starting fresh.")

# # ---- EVALUATION FUNCTION ----
# def evaluate_qa_pairs(pairs, description, csv_filename):
#     y_true, y_pred = [], []
#     all_preds, all_gts = [], []
#     bleu_scores, rougeL_scores, bart_scores = [], [], []

#     rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
#     bart_scorer = BARTScorer(device="cuda", checkpoint="facebook/bart-large-cnn")

#     with open(f"/kaggle/working/{csv_filename}", 'w', newline='') as csvfile:
#         fieldnames = ['json_file', 'question_id', 'question', 'ground_truth', 'predicted_answer']
#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#         writer.writeheader()

#         for json_file, idx, img_path, question, ground_truth in tqdm(pairs, desc="🔍 Evaluating"):
#             if (json_file, idx) in completed_set:
#                 continue

#             try:
#                 raw_image = Image.open(img_path).convert("RGB")
#                 inputs = processor(raw_image, question, return_tensors="pt").to("cuda")
#                 out = model.generate(**inputs)
#                 answer = processor.decode(out[0], skip_special_tokens=True)
#             except Exception as e:
#                 print(f"[ERROR] {json_file} Q{idx+1} → {e}")
#                 answer = "ERROR"

#             gt = ground_truth.strip().lower()
#             pred = answer.strip().lower()
#             y_true.append(gt)
#             y_pred.append(pred)
#             all_preds.append(pred)
#             all_gts.append(gt)

#             bleu_scores.append(sentence_bleu([gt.split()], pred.split()))
#             rougeL_scores.append(rouge.score(gt, pred)['rougeL'].fmeasure)
#             bart_scores.append(bart_scorer.score([pred], [gt])[0])

#             writer.writerow({
#                 'json_file': json_file,
#                 'question_id': idx,
#                 'question': question,
#                 'ground_truth': ground_truth,
#                 'predicted_answer': answer
#             })

#             completed_set.add((json_file, idx))
#             with open(progress_file, "w") as f:
#                 json.dump(list(completed_set), f)

#     # BERTScore (batched)
#     print("📏 Calculating BERTScore...")
#     P, R, F1 = bert_score(all_preds, all_gts, lang='en', verbose=True)
#     bert_P = P.numpy()
#     bert_R = R.numpy()
#     bert_F1 = F1.numpy()

#     acc = accuracy_score(y_true, y_pred)
#     f1 = f1_score(y_true, y_pred, average="macro")

#     print(f"\n✅ Evaluation complete: {description}")
#     print(f"Accuracy:       {acc:.4f}")
#     print(f"F1 Score:       {f1:.4f}")
#     print(f"BLEU:           {np.mean(bleu_scores):.4f}")
#     print(f"ROUGE-L:        {np.mean(rougeL_scores):.4f}")
#     print(f"BARTScore:      {np.mean(bart_scores):.4f}")
#     print(f"BERTScore (P/R/F1): {np.mean(bert_P):.4f} / {np.mean(bert_R):.4f} / {np.mean(bert_F1):.4f}")

# # ---- MAIN EXECUTION ----
# filtered_qa_data = [item for item in all_qa_data if (item[0], item[1]) not in completed_set]
# evaluate_qa_pairs(filtered_qa_data, "Full Dataset", "full_dataset_results1.csv")

In [ ]:
# import os
# import json
# import csv
# import pickle
# from PIL import Image
# from tqdm import tqdm
# from sklearn.metrics import accuracy_score, f1_score
# from nltk.translate.bleu_score import sentence_bleu
# from rouge_score import rouge_scorer
# from BARTScore.bart_score import BARTScorer
# from bert_score import score as bert_score
# import numpy as np
# import torch
# from transformers import Blip2Processor, Blip2ForConditionalGeneration

# # --- Define Paths ---
# json_folder_path = "/kaggle/input/test-dataset/test_dataset"
# image_base_path = "/kaggle/input/abo-small/small"
# all_qa_data_path = "/kaggle/working/all_qa_data1.pkl"
# progress_file = "/kaggle/working/progress1.json"

# # # --- Load BLIP-2 ---
# # processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
# # model = Blip2ForConditionalGeneration.from_pretrained(
# #     "Salesforce/blip2-opt-2.7b",
# #     device_map="auto",
# #     torch_dtype=torch.float16
# # )
# # model.eval()

# # ---- LOAD OR BUILD DATASET ----
# if os.path.exists(all_qa_data_path):
#     with open(all_qa_data_path, "rb") as f:
#         all_qa_data = pickle.load(f)
# else:
#     all_qa_data = []
#     for json_file in tqdm(os.listdir(json_folder_path), desc="📂 Loading JSONs"):
#         if not json_file.endswith(".json"):
#             continue
#         try:
#             with open(os.path.join(json_folder_path, json_file)) as f:
#                 data = json.load(f)
#         except:
#             continue

#         image_rel_path = "/".join(data["image_path"].replace("\\", "/").split("/")[-2:])
#         full_img_path = os.path.join(image_base_path, image_rel_path)
#         if not os.path.exists(full_img_path):
#             continue

#         for idx, qa in enumerate(data.get("qa_pairs", [])):
#             q, a = qa.get("question", "").strip(), qa.get("answer", "").strip()
#             if q and a:
#                 all_qa_data.append((json_file, idx, full_img_path, q, a))

#     with open(all_qa_data_path, "wb") as f:
#         pickle.dump(all_qa_data, f)

# # ---- LOAD PROGRESS ----
# completed_set = set()
# if os.path.exists(progress_file):
#     try:
#         with open(progress_file) as f:
#             completed_set = set(tuple(x) for x in json.load(f))
#     except:
#         pass

# # ---- BATCHED EVALUATION ----
# def evaluate_qa_pairs(pairs, description, csv_filename, batch_size=8):
#     y_true, y_pred, all_preds, all_gts = [], [], [], []
#     bleu_scores, rougeL_scores, bart_scores = [], [], []

#     rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
#     bart_scorer = BARTScorer(device="cuda", checkpoint="facebook/bart-large-cnn")

#     csv_path = f"/kaggle/working/{csv_filename}"
#     with open(csv_path, 'a', newline='') as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=["json_file", "question_id", "question", "ground_truth", "predicted_answer"])
#         writer.writeheader()

#         batch = []
#         for item in tqdm(pairs, desc="⚡ Batched Evaluation"):
#             if (item[0], item[1]) in completed_set:
#                 continue
#             batch.append(item)
#             if len(batch) == batch_size:
#                 process_batch(batch, writer, y_true, y_pred, all_preds, all_gts,
#                               bleu_scores, rougeL_scores, bart_scores, rouge, bart_scorer)
#                 batch = []
#         if batch:
#             process_batch(batch, writer, y_true, y_pred, all_preds, all_gts,
#                           bleu_scores, rougeL_scores, bart_scores, rouge, bart_scorer)

#     # Save progress
#     with open(progress_file, "w") as f:
#         json.dump(list(completed_set), f)

#     # BERTScore (batch)
#     print("📏 Calculating BERTScore...")
#     P, R, F1 = bert_score(all_preds, all_gts, lang='en', verbose=True)
#     acc = accuracy_score(y_true, y_pred)
#     f1 = f1_score(y_true, y_pred, average="macro")

#     print(f"\n✅ {description} Complete")
#     print(f"Accuracy:        {acc:.4f}")
#     print(f"F1 Score:        {f1:.4f}")
#     print(f"BLEU:            {np.mean(bleu_scores):.4f}")
#     print(f"ROUGE-L:         {np.mean(rougeL_scores):.4f}")
#     print(f"BARTScore:       {np.mean(bart_scores):.4f}")
#     print(f"BERTScore P/R/F: {P.mean():.4f} / {R.mean():.4f} / {F1.mean():.4f}")

# def process_batch(batch, writer, y_true, y_pred, all_preds, all_gts,
#                   bleu_scores, rougeL_scores, bart_scores, rouge, bart_scorer):
#     images, questions, metas = [], [], []
#     for json_file, idx, img_path, q, a in batch:
#         try:
#             img = Image.open(img_path).convert("RGB").resize((224, 224))
#         except:
#             continue
#         images.append(img)
#         questions.append(q)
#         metas.append((json_file, idx, q, a))

#     if not images:
#         return

#     inputs = processor(images=images, text=questions, return_tensors="pt", padding=True).to("cuda")
#     with torch.inference_mode():
#         outputs = model.generate(**inputs, max_new_tokens=30)
#     answers = processor.batch_decode(outputs, skip_special_tokens=True)

#     for ans, (json_file, idx, question, gt) in zip(answers, metas):
#         pred = ans.strip().lower()
#         gt_clean = gt.strip().lower()
#         all_preds.append(pred)
#         all_gts.append(gt_clean)
#         y_true.append(gt_clean)
#         y_pred.append(pred)

#         bleu_scores.append(sentence_bleu([gt_clean.split()], pred.split()))
#         rougeL_scores.append(rouge.score(gt_clean, pred)['rougeL'].fmeasure)
#         bart_scores.append(bart_scorer.score([pred], [gt_clean])[0])

#         writer.writerow({
#             "json_file": json_file,
#             "question_id": idx,
#             "question": question,
#             "ground_truth": gt,
#             "predicted_answer": ans
#         })
#         completed_set.add((json_file, idx))

# # ---- MAIN EXECUTION ----
# filtered_qa_data = [item for item in all_qa_data if (item[0], item[1]) not in completed_set]
# evaluate_qa_pairs(filtered_qa_data, "Full Dataset", "full_dataset_results1.csv", batch_size=8)


In [ ]:
# import csv
# from bert_score import score as bert_score
# import numpy as np

# # --- CONFIG ---
# csv_path = "/kaggle/working/full_dataset_results.csv"  # or any result file
# bert_lang = "en"  # language for BERTScore

# # --- LOAD CSV DATA ---
# ground_truths = []
# predictions = []

# print(f"📂 Reading predictions from {csv_path}...")
# with open(csv_path, 'r') as f:
#     reader = csv.DictReader(f)
#     for row in reader:
#         gt = row['ground_truth'].strip()
#         pred = row['predicted_answer'].strip()
#         if gt and pred:
#             ground_truths.append(gt)
#             predictions.append(pred)

# print(f"✅ Loaded {len(ground_truths)} Q&A pairs.")

# # --- COMPUTE BERTSCORE ---
# print("🔍 Computing BERTScore...")
# P, R, F1 = bert_score(predictions, ground_truths, lang=bert_lang, verbose=True)
# P = P.numpy()
# R = R.numpy()
# F1 = F1.numpy()
# # --- OUTPUT RESULTS ---
# print("\n🎯 BERTScore Results:")
# print(f"Precision:  {np.mean(P):.4f}")
# print(f"Recall:     {np.mean(R):.4f}")
# print(f"F1 Score:   {np.mean(F1):.4f}")

# # --- OPTIONAL: SAVE DETAILED SCORES ---
# with open("/kaggle/working/bertscore_details.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Ground Truth", "Prediction", "Precision", "Recall", "F1"])
#     for gt, pred, p, r, f1 in zip(ground_truths, predictions, P, R, F1):
#         writer.writerow([gt, pred, f"{p:.4f}", f"{r:.4f}", f"{f1:.4f}"])

# print("📁 Saved detailed BERTScore results to bertscore_details.csv")

In [19]:
import os
import json
import csv
import pickle
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import accuracy_score, f1_score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from BARTScore.bart_score import BARTScorer
from bert_score import score as bert_score
import numpy as np
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
# --- Define Paths ---
json_folder_path = "/kaggle/input/master-test/test_dataset"
image_base_path = "/kaggle/input/abo-small/small"
all_qa_data_path = "/kaggle/working/all_qa_data1.pkl"
progress_file = "/kaggle/working/progress1.json"
csv_path = "/kaggle/working/full_dataset_results1.csv"

# --- Load BLIP-2 ---
# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
# model = Blip2ForConditionalGeneration.from_pretrained(
#     "Salesforce/blip2-opt-2.7b",
#     device_map="auto",
#     torch_dtype=torch.float16
# )
# model.eval()

# ---- LOAD OR BUILD DATASET ----
if os.path.exists(all_qa_data_path):
    with open(all_qa_data_path, "rb") as f:
        all_qa_data = pickle.load(f)
else:
    all_qa_data = []
    for json_file in tqdm(os.listdir(json_folder_path), desc="📂 Loading JSONs"):
        if not json_file.endswith(".json"):
            continue
        try:
            with open(os.path.join(json_folder_path, json_file)) as f:
                data = json.load(f)
        except:
            continue

        image_rel_path = "/".join(data["image_path"].replace("\\", "/").split("/")[-2:])
        full_img_path = os.path.join(image_base_path, image_rel_path)
        if not os.path.exists(full_img_path):
            continue

        for idx, qa in enumerate(data.get("qa_pairs", [])):
            q, a = qa.get("question", "").strip(), qa.get("answer", "").strip()
            if q and a:
                all_qa_data.append((json_file, idx, full_img_path, q, a))

    with open(all_qa_data_path, "wb") as f:
        pickle.dump(all_qa_data, f)

# ---- LOAD PROGRESS ----
completed_set = set()
if os.path.exists(progress_file):
    try:
        with open(progress_file) as f:
            completed_set = set(tuple(x) for x in json.load(f))
    except:
        pass

# ---- BATCH PROCESSING UTILS ----
def load_image_resize(path):
    try:
        return Image.open(path).convert("RGB").resize((224, 224))
    except:
        return None

def process_batch(batch, writer, y_true, y_pred, all_preds, all_gts,
                  bleu_scores, rougeL_scores, bart_scores, rouge, bart_scorer):
    metas = [(json_file, idx, img_path, q, a) for json_file, idx, img_path, q, a in batch]
    questions = [q for _, _, _, q, _ in metas]

    with ThreadPoolExecutor(max_workers=8) as executor:
        images = list(executor.map(lambda x: load_image_resize(x[2]), metas))

    valid_data = [(img, meta) for img, meta in zip(images, metas) if img is not None]
    if not valid_data:
        return

    images = [img for img, _ in valid_data]
    metas = [meta for _, meta in valid_data]
    questions = [q for _, _, _, q, _ in metas]

    inputs = processor(images=images, text=questions, return_tensors="pt", padding=True).to("cuda")
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=30)
    answers = processor.batch_decode(outputs, skip_special_tokens=True)

    for ans, (json_file, idx, _, question, gt) in zip(answers, metas):
        pred = ans.strip().lower()
        gt_clean = gt.strip().lower()
        all_preds.append(pred)
        all_gts.append(gt_clean)
        y_true.append(gt_clean)
        y_pred.append(pred)
    
        bleu_scores.append(sentence_bleu([gt_clean.split()], pred.split()))
        rougeL_scores.append(rouge.score(gt_clean, pred)['rougeL'].fmeasure)
        bart_scores.append(bart_scorer.score([pred], [gt_clean])[0])
    
        writer.writerow({
            "json_file": json_file,
            "question_id": idx,
            "question": question,
            "ground_truth": gt,
            "predicted_answer": ans
        })
        completed_set.add((json_file, idx))

    # Save progress after each batch
    with open(progress_file, "w") as f:
        json.dump(list(completed_set), f)

# ---- MAIN EVALUATION FUNCTION ----
def evaluate_qa_pairs(pairs, description, batch_size=64):
    y_true, y_pred, all_preds, all_gts = [], [], [], []
    bleu_scores, rougeL_scores, bart_scores = [], [], []

    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    bart_scorer = BARTScorer(device="cuda", checkpoint="facebook/bart-large-cnn")

    write_header = not os.path.exists(csv_path)
    with open(csv_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["json_file", "question_id", "question", "ground_truth", "predicted_answer"])
        if write_header:
            writer.writeheader()
    
        batch = []
        with tqdm(total=len(pairs), desc="⚡ Batched Evaluation", dynamic_ncols=True) as pbar:
            for item in pairs:
                if (item[0], item[1]) in completed_set:
                    pbar.update(1)  # Update progress bar by 1 for skipped item
                    continue
                batch.append(item)
                if len(batch) == batch_size:
                    process_batch(batch, writer, y_true, y_pred, all_preds, all_gts,
                                  bleu_scores, rougeL_scores, bart_scores, rouge, bart_scorer)
                    pbar.update(len(batch))  # Update progress after processing a full batch
                    batch = []
            
            # Process any remaining items in the batch
            if batch:
                process_batch(batch, writer, y_true, y_pred, all_preds, all_gts,
                              bleu_scores, rougeL_scores, bart_scores, rouge, bart_scorer)
                pbar.update(len(batch))  # Update progress after the last batch
    
            pbar.close()  # Close the progress bar after processing all batches



    print("📏 Calculating BERTScore...")
    P, R, F1 = bert_score(all_preds, all_gts, lang='en', verbose=True)
    P = P.numpy()
    R = R.numpy()
    F1 = F1.numpy()
    
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")

    print(f"\n✅ {description} Complete")
    print(f"Accuracy:        {acc:.4f}")
    print(f"F1 Score:        {f1:.4f}")
    print(f"BLEU:            {np.mean(bleu_scores):.4f}")
    print(f"ROUGE-L:         {np.mean(rougeL_scores):.4f}")
    print(f"BARTScore:       {np.mean(bart_scores):.4f}")
    print(f"BERTScore P/R/F: {P.mean():.4f} / {R.mean():.4f} / {F1.mean():.4f}")

# ---- EXECUTION ----
filtered_qa_data = [item for item in all_qa_data if (item[0], item[1]) not in completed_set]
evaluate_qa_pairs(filtered_qa_data, "Full Dataset", batch_size=64)

⚡ Batched Evaluation: 0it [00:00, ?it/s]

📏 Calculating BERTScore...


IndexError: list index out of range

In [22]:
import pandas as pd
from bert_score import score as bert_score
from sklearn.metrics import accuracy_score, f1_score

# Load your CSV
df = pd.read_csv("/kaggle/working/full_dataset_results1.csv")
df.columns = df.columns.str.strip().str.lower()  # normalize column names

# Extract predictions and ground truths
all_preds = df['predicted_answer'].astype(str).str.strip().str.lower().tolist()
all_gts = df['ground_truth'].astype(str).str.strip().str.lower().tolist()


# 🎯 Exact Match Accuracy (using sklearn)
acc = accuracy_score(all_gts, all_preds)
f1_exact = f1_score(all_gts, all_preds, average='macro')  # or 'weighted'

print(f"✅ Accuracy (sklearn exact match): {acc:.4f}")
print(f"🎯 F1 Score (exact match, macro): {f1_exact:.4f}")

# 🧠 BERTScore (semantic similarity)
print("📏 Calculating BERTScore...")
P, R, F1 = bert_score(all_preds, all_gts, lang='en', verbose=True)
print(f"\n🔍 BERTScore (avg):\nPrecision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")


✅ Accuracy (sklearn exact match): 0.1768
🎯 F1 Score (exact match, macro): 0.0076
📏 Calculating BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/557 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/7532 [00:00<?, ?it/s]

done in 120.20 seconds, 4010.23 sentences/sec

🔍 BERTScore (avg):
Precision: 0.9408, Recall: 0.9310, F1: 0.9350
